In [1]:
import os 
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN-TYPE', '')

## Import and EDA

In [2]:
from pathlib import Path

# Directorio del dataset
path = Path('../input/us-patent-phrase-to-phrase-matching')

# Instalar el paquete datasets
! pip install -q datasets

In [3]:
!ls {path}

sample_submission.csv  test.csv  train.csv


### Import libraries

In [45]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv(path/"train.csv")

In [6]:
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [7]:
# adding 'object' describing string fields
df.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,8d135da0b55b8c88,component composite coating,composition,H01
freq,1,152,24,2186


* There are 36472 rows, 733 unique anchor, 106 contexts and nearly 30000 targets. 
* Represent input to the model as the following
    * "TEXT1: abatement; TEXT2: eliminating process" 

In [8]:
df['input'] = 'TEXT1: ' + df.context + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor

In [9]:
df.input.head()

0    TEXT1: A47; TEXT2: abatement of pollution; ANC...
1    TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2    TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3    TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4    TEXT1: A47; TEXT2: forest region; ANC1: abatement
Name: input, dtype: object

## Tokenization

Transformers use a `Dataset` object storing a dataset, so we create one like: 

In [10]:
from datasets import Dataset, DatasetDict

ds = Dataset.from_pandas(df)

In [11]:
print(type(ds))
ds

<class 'datasets.arrow_dataset.Dataset'>


Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

#### Tokenization: split each text up into words
#### Numericalization: convert each word/token into a number
##### Big vocabulary requires more capacity and memory so we aim to limit the vocabulary

* Replace 'small' with 'large' for slower but more accurate pretrained model 

In [12]:
model_nm = 'microsoft/deberta-v3-small'

`AutoTokenizer` will create a tokenizer appropriate for a given model:

In [13]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokz = AutoTokenizer.from_pretrained(model_nm)    # download the vocabulary and details of how the model tokenizes the dataset

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [14]:
tokz.tokenize("G'day folks, I'm Tinkerbell from fast.ai")

['▁G',
 "'",
 'day',
 '▁folks',
 ',',
 '▁I',
 "'",
 'm',
 '▁Tinkerbell',
 '▁from',
 '▁fast',
 '.',
 'ai']

* '▁G': underscore indicate start of the word

#### Helper function to tokenize our inputs

In [15]:
def tok_func(x):
    return tokz(x['input'])

Use `map` to run this quickly in parallel on every row in our dataset:

In [16]:
tok_ds = ds.map(tok_func, batched=True)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

This adds a new item to our dataset called `input_ids`. For instance, here's the input and ids for the 1st row:

In [17]:
row = tok_ds[0]
row['input'], row['input_ids']

('TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement',
 [1,
  54453,
  435,
  294,
  336,
  5753,
  346,
  54453,
  445,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  435,
  294,
  47284,
  2])

Check the ids of 1 example tokenized string

In [21]:
tokz.vocab['of']

1580

Prepare labels. Transformers always assume that your labels has the column name `labels`, but in our dataset it's currently `score`. Therefore, we need to rename it:

In [23]:
tok_ds = tok_ds.rename_columns({'score': 'labels'})

## Test and validation sets

In [25]:
eval_df = pd.read_csv(path/'test.csv')
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,hybrid bearing,inorganic photoconductor drum,G02
freq,1,2,1,3


Transformers expect metrics to be returned as a `dict`, since that way the trainer knows what labels to use, so we create a function to do that:

### Split data

In [28]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [49]:
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [40]:
def corr(x,y): 
    return np.corrcoef(x,y)[0][1]

In [41]:
def corr_d(eval_pred):
    return {'pearson':  corr(*eval_pred)}

## Train our model

In [29]:
from transformers import TrainingArguments, Trainer

2024-06-10 17:42:10.785907: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-10 17:42:10.786041: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-10 17:42:11.130041: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [30]:
# bs = batch size
bs = 128
epochs = 4

Set learning rate

In [31]:
lr = 8e-5

Transformers use the `TrainingArguments` class to set up arguments

In [34]:
# this is similar to fit_one_cylc in fast_ai
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
                        evaluation_strategy='epoch', per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
                        num_train_epochs=epochs, weight_decay=0.01, report_to='none')   

Create our model and `Trainer`, which is a class that combines the data and model together (like `Learner` in fastai)

In [42]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)

trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'], 
                 tokenizer=tokz, compute_metrics=corr_d)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
trainer.train();

Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.021302,0.825315
2,No log,0.021642,0.831314
3,0.015400,0.022742,0.837526
4,0.015400,0.021776,0.837906


Get predictions on test set

In [50]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

array([[ 0.42138672],
       [ 0.67041016],
       [ 0.55224609],
       [ 0.29125977],
       [-0.03152466],
       [ 0.47729492],
       [ 0.51513672],
       [-0.02864075],
       [ 0.26831055],
       [ 1.07324219],
       [ 0.26342773],
       [ 0.24963379],
       [ 0.76757812],
       [ 0.953125  ],
       [ 0.77050781],
       [ 0.3984375 ],
       [ 0.29296875],
       [-0.03076172],
       [ 0.56445312],
       [ 0.3671875 ],
       [ 0.49365234],
       [ 0.22949219],
       [ 0.22436523],
       [ 0.26708984],
       [ 0.57324219],
       [-0.02163696],
       [-0.0369873 ],
       [-0.02693176],
       [-0.02770996],
       [ 0.73535156],
       [ 0.33666992],
       [-0.03305054],
       [ 0.71875   ],
       [ 0.47998047],
       [ 0.31982422],
       [ 0.23339844]])

Some predictions are < 0 and > 1 so we need to fix these out-of-bound predictions

In [51]:
# clip makes any < 0 to 0 and any > 1 to 1
preds = np.clip(preds, 0, 1)

In [52]:
preds

array([[0.42138672],
       [0.67041016],
       [0.55224609],
       [0.29125977],
       [0.        ],
       [0.47729492],
       [0.51513672],
       [0.        ],
       [0.26831055],
       [1.        ],
       [0.26342773],
       [0.24963379],
       [0.76757812],
       [0.953125  ],
       [0.77050781],
       [0.3984375 ],
       [0.29296875],
       [0.        ],
       [0.56445312],
       [0.3671875 ],
       [0.49365234],
       [0.22949219],
       [0.22436523],
       [0.26708984],
       [0.57324219],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.73535156],
       [0.33666992],
       [0.        ],
       [0.71875   ],
       [0.47998047],
       [0.31982422],
       [0.23339844]])

## Create submission

In [54]:
import datasets

submission = datasets.Dataset.from_dict({
    'id': eval_ds['id'],
    'score': preds
})

submission.to_csv('submission.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1010